In [6]:
import numpy as np
import pandas as pd

In [7]:
games_df = pd.read_csv('game_data.csv', index_col=0)
games_df.head()

C:\Users\grego\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,char1,char2,stage,winner,entrant1,entrant2
0,Isabelle,Simon Belmont,Pokémon Stadium 2,Simon Belmont,ALS/UNF/PHL | Angelseb,RS | Kuro Dan
1,Simon Belmont,Isabelle,Pokémon Stadium 2,Simon Belmont,RS | Kuro Dan,ALS/UNF/PHL | Angelseb
2,King K. Rool,Snake,Pokémon Stadium 2,Snake,Siux,TB PHL | SrJorjoroY
3,Little Mac,King K. Rool,Final Destination,Little Mac,TB PHL | SrJorjoroY,Siux
4,Simon Belmont,Snake,Smashville,Snake,RS | Kuro Dan,TB PHL | SrJorjoroY


In [8]:
games_df['stage'].value_counts()

Pokémon Stadium 2       667189
Town and City           266913
Final Destination       263196
Battlefield             245498
Smashville              242004
Kalos Pokémon League    134777
Small Battlefield       119113
Yoshi's Story            64100
Lylat Cruise             30723
Yoshi's Island            7614
Pokémon Stadium           2542
Unova Pokémon League      2079
Castle Siege              1379
Northern Cave              650
WarioWare, Inc.            351
Frigate Orpheon             74
Skyloft                     55
Magicant                    23
Mario Circuit               12
Mementos                     7
Yggdrasil’s Altar            1
Name: stage, dtype: int64

In [9]:
char_totals = games_df['char1'].value_counts() + games_df['char2'].value_counts()
sorted_totals = char_totals.sort_values(ascending=False)
for count, char in zip(sorted_totals, sorted_totals.index):
    print('{}\t\t\t{}'.format(char, count))

Ness			115516
Palutena			110516
Bowser			109192
Joker			104520
Snake			101783
Wolf			101607
R.O.B.			100102
Pokemon Trainer			97996
Cloud			95846
Inkling			88639
Yoshi			85387
Donkey Kong			82602
Mario			75223
Steve			74192
King Dedede			74061
Luigi			72106
Captain Falcon			71751
Link			71633
Sonic			69959
King K. Rool			69497
Terry			69405
Pyra & Mythra			67559
Ike			65826
Mr. Game and Watch			65514
Lucina			65068
Zelda			59899
Jigglypuff			59858
Samus			59680
Pikachu			59390
Banjo-Kazooie			57360
Pac-Man			56090
Min Min			55952
Hero			54767
Ganondorf			54275
Sephiroth			52617
Young Link			50758
Wii Fit Trainer			49640
Zero Suit Samus			49361
Roy			49253
Lucas			49144
Little Mac			48861
Falco			45647
Greninja			45458
Bayonetta			43566
Kirby			43438
Toon Link			39410
Incineroar			39345
Byleth			38243
Robin			37987
Piranha Plant			37369
Ken			37182
Diddy Kong			37013
Bowser Jr.			36188
Random			36176
Ridley			33432
Fox			33290
Chrom			33093
Mega Man			32809
Shulk			32573
Pichu			30691
I

In [10]:
char_to_index = {}

i = 0
for char in games_df['char1']:
    if char not in char_to_index:
        char_to_index[char] = i
        i += 1
print(len(char_to_index))

# format = winner: loser: count
wins_map = {}

for _, row in games_df.iterrows():
    char1 = row['char1']
    char2 = row['char2']
    
    char1_is_winner = row['char1'] == row['winner']
    
    if not char1_is_winner:
        char1, char2 = (char2, char1)
        
    if char1 not in wins_map:
        wins_map[char1] = {}
        wins_map[char1][char2] = 1
    elif char2 not in wins_map[char1]:
        wins_map[char1][char2] = 1
    else:
        wins_map[char1][char2] += 1
matchup_info = {'char1': [], 
                'char2': [],
                'win_rate': [],
                'games_played': []
               }
char_to_win_rate = {}
for char1 in wins_map:
    char_win_rates = []
    for char2 in wins_map[char1]:
        matchup_info['char1'].append(char1)
        matchup_info['char2'].append(char2)
        
        games_played = wins_map[char1][char2]
        try:
            games_played += wins_map[char2][char1]
        except KeyError:
            pass
        
        win_rate = wins_map[char1][char2] / games_played
        char_win_rates.append(win_rate)
        
        matchup_info['win_rate'].append(win_rate)
        matchup_info['games_played'].append(games_played)
    char_to_win_rate[char1] = sum(char_win_rates) / len(char_win_rates)
    
for key in matchup_info:
    print(key, len(matchup_info[key]))
matchup_df = pd.DataFrame.from_dict(matchup_info)

matchup_df

86
char1 7499
char2 7499
win_rate 7499
games_played 7499


,char1,char2,win_rate,games_played
0,Simon Belmont,Isabelle,0.551724,116
1,Simon Belmont,Pikachu,0.517857,168
2,Simon Belmont,Link,0.553991,213
3,Simon Belmont,Hero,0.653061,147
4,Simon Belmont,Ness,0.591398,279
...,...,...,...,...
7494,Kazuya,Ryu,0.333333,33
7495,Kazuya,Lucario,0.312500,16
7496,Kazuya,Chrom,0.440000,25
7497,Kazuya,Daisy,0.500000,6


In [11]:
#matchup_df.to_csv('matchups.csv')

In [12]:
clean_matchup_df = matchup_df[matchup_df['char1'] != 'Random']
clean_matchup_df = clean_matchup_df[clean_matchup_df['char2'] != 'Random']
clean_matchup_df = clean_matchup_df[clean_matchup_df['games_played'] > 10]

clean_matchup_df

,char1,char2,win_rate,games_played
0,Simon Belmont,Isabelle,0.551724,116
1,Simon Belmont,Pikachu,0.517857,168
2,Simon Belmont,Link,0.553991,213
3,Simon Belmont,Hero,0.653061,147
4,Simon Belmont,Ness,0.591398,279
...,...,...,...,...
7493,Kazuya,Fox,0.481481,27
7494,Kazuya,Ryu,0.333333,33
7495,Kazuya,Lucario,0.312500,16
7496,Kazuya,Chrom,0.440000,25


In [13]:
clean_matchup_df = matchup_df[matchup_df['char1'] != 'Random']
clean_matchup_df = clean_matchup_df[clean_matchup_df['char2'] != 'Random']
clean_matchup_df = clean_matchup_df[clean_matchup_df['games_played'] > 50].sort_values('win_rate', ascending=False).head(10)

clean_matchup_df

,char1,char2,win_rate,games_played
5032,Pokemon Trainer,NaN,1.0,323
1716,Peach,NaN,1.0,76
2504,Pikachu,NaN,1.0,169
1775,King K. Rool,NaN,1.0,224
3025,Terry,NaN,1.0,175
3243,Ken,NaN,1.0,112
3512,Corrin,NaN,1.0,90
6155,Piranha Plant,NaN,1.0,130
3342,Ike,NaN,1.0,274
7169,Mii Brawler,NaN,1.0,107


In [14]:
clean_matchup_df[clean_matchup_df['games_played'] > 1000].sort_values('win_rate', ascending=False).head(84)

,char1,char2,win_rate,games_played


In [15]:

stages_df = games_df.dropna()

In [16]:
stages_df.head()

,char1,char2,stage,winner,entrant1,entrant2
0,Isabelle,Simon Belmont,Pokémon Stadium 2,Simon Belmont,ALS/UNF/PHL | Angelseb,RS | Kuro Dan
1,Simon Belmont,Isabelle,Pokémon Stadium 2,Simon Belmont,RS | Kuro Dan,ALS/UNF/PHL | Angelseb
2,King K. Rool,Snake,Pokémon Stadium 2,Snake,Siux,TB PHL | SrJorjoroY
3,Little Mac,King K. Rool,Final Destination,Little Mac,TB PHL | SrJorjoroY,Siux
4,Simon Belmont,Snake,Smashville,Snake,RS | Kuro Dan,TB PHL | SrJorjoroY


In [17]:
legal_stages = set(['Pokémon Stadium 2', 'Smashville', 'Battlefield', 'Final Destination',
                    'Kalos Pokémon League', 'Lylat Cruise', 'Town and City',
                    "Yoshi's Story", 'Small Battlefield'
                   ])

stage_wins = {}
stage_losses = {}

for _, row in stages_df.iterrows():
    char1 = row['char1']
    char2 = row['char2']
    winner = row['winner']
    stage = row['stage']
    
    if stage not in legal_stages:
        continue
    
    if char1 == char2: 
        continue
        
    if char1 != winner:
        char1, char2 = (char2, char1)
    
    if stage in stage_wins:
        if char1 in stage_wins[stage]:
            stage_wins[stage][char1] += 1
        else:
            stage_wins[stage][char1] = 1
    else:
        stage_wins[stage] = {}
        stage_wins[stage][char1] = 1
            
    if stage in stage_losses:
        if char2 in stage_losses[stage]:
            stage_losses[stage][char2] += 1
        else:
            stage_losses[stage][char2] = 1
    else:
        stage_losses[stage] = {}
        stage_losses[stage][char2] = 1

In [18]:
winrate_info = {'char': [], 'stage': [], 'character_winrate': [],
                    'stage_winrate': [], 'games_played': []}

for stage in stage_wins:
    for char in stage_wins[stage]:
        if stage not in stage_losses or char not in stage_losses[stage]:
            continue
        wins = stage_wins[stage][char]
        losses = stage_losses[stage][char]
        win_rate = wins / (wins + losses)
        
        winrate_info['char'].append(char)
        winrate_info['stage'].append(stage)
        winrate_info['stage_winrate'].append(win_rate)
        winrate_info['games_played'].append(wins + losses)
        winrate_info['character_winrate'].append(char_to_win_rate[char])

In [19]:
char_winrate = {'char':[],'char_winrate':[]}

all_chars = set()

for stage in stage_wins:
    for char in stage_wins[stage]:
        if stage not in stage_losses or char not in stage_losses[stage]:
            continue
        wins = stage_wins[stage][char]
        losses = stage_losses[stage][char]
        win_rate = wins / (wins + losses)
        
        if char not in all_chars:
            all_chars.add(char)
            char_winrate['char'].append(char)
            char_winrate['char_winrate'].append(char_to_win_rate[char])

        

char_winrate = pd.DataFrame.from_dict(char_winrate)

char_winrate.sort_values(by='char_winrate',ascending=False).to_csv('character_winrate.csv')


In [20]:

winrate_df = pd.DataFrame.from_dict(winrate_info)

In [21]:
winrate_df['stage_benefit'] = winrate_df['stage_winrate'] - winrate_df['character_winrate'] 
winrate_df

,char,stage,character_winrate,stage_winrate,games_played,stage_benefit
0,Simon Belmont,Pokémon Stadium 2,0.584877,0.579771,3579,-0.005106
1,Snake,Pokémon Stadium 2,0.584781,0.581100,35068,-0.003682
2,Isabelle,Pokémon Stadium 2,0.534786,0.509666,7811,-0.025120
3,Bowser,Pokémon Stadium 2,0.501319,0.490479,30825,-0.010841
4,Samus,Pokémon Stadium 2,0.535840,0.536321,19369,0.000481
...,...,...,...,...,...,...
769,Mewtwo,Lylat Cruise,0.445621,0.391489,235,-0.054132
770,Rosalina,Lylat Cruise,0.473285,0.402439,164,-0.070846
771,Sephiroth,Lylat Cruise,0.436397,0.407328,464,-0.029070
772,Pyra & Mythra,Lylat Cruise,0.470971,0.487842,658,0.016871


In [ ]:
for char1 in zip(sorted_totals, sorted_totals.index):
    for char2 in zip(sorted_totals, sorted_totals.index):
        char_stage_benifit = {'char1':[], 'char2':[], 'stage':[], 'stage_benifit':[]}
        if char1 != char2:
            char_stage_benifit['char1'].append(char1)
            char_stage_benifit['char2'].append(char2)


In [22]:
#winrate_df.to_csv('winrate1.csv')

In [23]:
winrate_df.sort_values('stage_benefit')

,char,stage,character_winrate,stage_winrate,games_played,stage_benefit
336,Kirby,Battlefield,0.459657,0.348411,1857,-0.111245
768,Bowser Jr.,Lylat Cruise,0.555591,0.446328,177,-0.109264
698,Ice Climbers,Lylat Cruise,0.522865,0.422360,161,-0.100504
601,Kazuya,Yoshi's Story,0.484174,0.387500,80,-0.096674
687,Kazuya,Small Battlefield,0.484174,0.397924,289,-0.086251
...,...,...,...,...,...,...
716,Duck Hunt,Lylat Cruise,0.551293,0.596535,404,0.045242
765,Sheik,Lylat Cruise,0.417875,0.466667,150,0.048791
557,Dark Samus,Yoshi's Story,0.551091,0.610454,1014,0.059363
583,Ice Climbers,Yoshi's Story,0.522865,0.584016,488,0.061152


In [24]:
#winrate_df.sort_values('stage_benefit').to_csv('winrate2.csv')

In [25]:
winrate_df[winrate_df['char'] == 'Yoshi']

,char,stage,character_winrate,stage_winrate,games_played,stage_benefit
59,Yoshi,Pokémon Stadium 2,0.519926,0.507940,25692,-0.011986
115,Yoshi,Final Destination,0.519926,0.523317,10250,0.003391
218,Yoshi,Smashville,0.519926,0.503570,9383,-0.016356
280,Yoshi,Battlefield,0.519926,0.505575,11122,-0.014352
381,Yoshi,Kalos Pokémon League,0.519926,0.538629,5061,0.018702
461,Yoshi,Town and City,0.519926,0.501631,9503,-0.018295
552,Yoshi,Yoshi's Story,0.519926,0.480955,3308,-0.038971
637,Yoshi,Small Battlefield,0.519926,0.508471,4604,-0.011455
703,Yoshi,Lylat Cruise,0.519926,0.514845,1583,-0.005081


In [26]:
#winrate_df[winrate_df['stage'] == 'Pokémon Stadium 2'].sort_values('character_winrate', ascending=False).to_csv('Char_WinRates.csv')

In [27]:
# winrate_df[winrate_df['stage'] == 'Pokémon Stadium 2'].to_csv('Pokémon Stadium 2.csv')
# winrate_df[winrate_df['stage'] == 'Town and City'].to_csv('Town and City.csv')
# winrate_df[winrate_df['stage'] == 'Final Destination'].to_csv('Final Destination.csv')
# winrate_df[winrate_df['stage'] == 'Battlefield'].to_csv('Battlefield.csv')
# winrate_df[winrate_df['stage'] == 'Smashville'].to_csv('Smashville.csv')
# winrate_df[winrate_df['stage'] == 'Kalos Pokémon League'].to_csv('Kalos Pokémon League.csv')
# winrate_df[winrate_df['stage'] == 'Small Battlefield'].to_csv('Small Battlefield.csv')
# winrate_df[winrate_df['stage'] == "Yoshi's Story"].to_csv("Yoshi's Story.csv")
# winrate_df[winrate_df['stage'] == 'Lylat Cruise'].to_csv('Lylat Cruise.csv')

In [28]:
# for char in zip(sorted_totals, sorted_totals.index):
#     csv = '{}.csv'.format(char[1])
#     winrate_df[winrate_df['char'] == char[1]].sort_values('stage_benefit').to_csv(csv)